<a href="https://colab.research.google.com/github/DuongTrucUyen/IS403.O21.HTCL_Nhom11/blob/main/BNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM
from sklearn.preprocessing import MinMaxScaler

df=pd.read_csv("/content/BNB.csv")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,3/1/2019,10.356931,11.607489,10.325634,11.393217,11.393217,117838677
1,3/2/2019,11.384693,11.915753,10.960816,11.869744,11.869744,120807894
2,3/3/2019,11.845305,11.964979,11.352064,11.537903,11.537903,89868246
3,3/4/2019,11.538013,11.599405,11.070543,11.545945,11.545945,89421231
4,3/5/2019,11.520388,13.841443,11.520388,13.811967,13.811967,237026335
...,...,...,...,...,...,...,...
1899,5/12/2024,592.029175,599.184143,587.804321,594.424560,594.424560,1276671115
1900,5/13/2024,594.424560,597.768005,585.520752,591.606934,591.606934,1636169234
1901,5/14/2024,591.606934,592.836975,563.420227,566.707031,566.707031,1944063031
1902,5/15/2024,566.707031,587.421509,561.127258,582.074341,582.074341,1895100260


SEMOS

In [ ]:

# Preprocessing: Convert 'Date' to datetime and sort the dataset by date
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Define the features and target variable
X = df[['Open', 'High', 'Low', 'Volume']]
y = df['Close']

# Dictionary to store results
results = {}
# Define train-test ratios
ratios = {'SEMOS 8-2': 0.8, 'SEMOS 7-3': 0.7, 'SEMOS 6-4': 0.6}

for ratio_name, ratio in ratios.items():
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=ratio, shuffle=False)

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    results[ratio_name] = mse

results


{'SEMOS 8-2': 20.65541639582039,
 'SEMOS 7-3': 18.922140843993336,
 'SEMOS 6-4': 20.965780835132744}

Linear Regression, ARIMA, RNN, GRU, LSTM

In [ ]:

# Function to evaluate ARIMA
def evaluate_arima(train, test, order=(5, 1, 0)):
    history = [x for x in train]
    predictions = []
    for t in range(len(test)):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    mse = mean_squared_error(test, predictions)
    return mse

# Function to create RNN, GRU, LSTM models
def create_model(model_type, input_shape):
    model = Sequential()
    if model_type == 'RNN':
        model.add(SimpleRNN(50, activation='relu', input_shape=input_shape))
    elif model_type == 'GRU':
        model.add(GRU(50, activation='relu', input_shape=input_shape))
    elif model_type == 'LSTM':
        model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Function to preprocess and evaluate RNN, GRU, LSTM
def evaluate_rnn_gru_lstm(model_type, X_train, y_train, X_test, y_test, epochs=50):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    model = create_model(model_type, (X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
    model.fit(X_train_reshaped, y_train, epochs=epochs, verbose=0)
    y_pred = model.predict(X_test_reshaped)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Update results dictionary with new algorithms
for ratio_name, ratio in ratios.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=ratio, shuffle=False)

    # ARIMA
    arima_mse = evaluate_arima(y_train.values, y_test.values)
    results[f'ARIMA {ratio_name}'] = arima_mse

    # RNN
    rnn_mse = evaluate_rnn_gru_lstm('RNN', X_train, y_train, X_test, y_test)
    results[f'RNN {ratio_name}'] = rnn_mse

    # GRU
    gru_mse = evaluate_rnn_gru_lstm('GRU', X_train, y_train, X_test, y_test)
    results[f'GRU {ratio_name}'] = gru_mse

    # LSTM
    lstm_mse = evaluate_rnn_gru_lstm('LSTM', X_train, y_train, X_test, y_test)
    results[f'LSTM {ratio_name}'] = lstm_mse

results


24/24 [==============================] - 0s 2ms/step


{'8-2': 20.65541639582039,
 '7-3': 18.922140843993336,
 '6-4': 20.965780835132744,
 'ARIMA 8-2': 131.20666201160293,
 'RNN 8-2': 3791.9654660056635,
 'GRU 8-2': 983.6056827967627,
 'LSTM 8-2': 1189.438601124197,
 'ARIMA 7-3': 117.01755512576749,
 'RNN 7-3': 2540.281613072126,
 'GRU 7-3': 1540.4608364050566,
 'LSTM 7-3': 2066.929220552539,
 'ARIMA 6-4': 116.01803436479604,
 'RNN 6-4': 6220.609506960408,
 'GRU 6-4': 4477.458703276044,
 'LSTM 6-4': 1595.0721794792678}